In [1]:
def apply_async(func, args, *, callback):
    """
    Simulates an asynchronous operation by computing the result of `func(*args)`
    and then invoking the `callback` with the result.

    :param func: The function to execute asynchronously.
    :param args: A tuple of arguments to pass to `func`.
    :param callback: A callback function that accepts the result of `func(*args)`.
    """
    # Compute the result synchronously (for simplicity)
    result = func(*args)

    # Invoke the callback with the result
    callback(result)

In [3]:
class ResultHandler:
    def __init__(self):
        self.sequence = 0

    def handler(self, result):
        self.sequence += 1
        print(f'[{self.sequence}] Got: {result}')

# Create an instance of ResultHandler
r = ResultHandler()

def add(x, y):
    return x + y

# Use apply_async with the bound method as the callback
apply_async(add, (2, 3), callback=r.handler)
apply_async(add, ('hello', 'world'), callback=r.handler)

[1] Got: 5
[2] Got: helloworld


In [4]:
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print(f'[{sequence}] Got: {result}')
    return handler

# Create a handler using the closure
handler = make_handler()

# Use apply_async with the closure as the callback
apply_async(add, (2, 3), callback=handler)
apply_async(add, ('hello', 'world'), callback=handler)

[1] Got: 5
[2] Got: helloworld


In [5]:
def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print(f'[{sequence}] Got: {result}')

# Create a coroutine handler
handler = make_handler()
next(handler)  # Advance to the yield

# Use apply_async with the coroutine's send method as the callback
apply_async(add, (2, 3), callback=handler.send)
apply_async(add, ('hello', 'world'), callback=handler.send)

[1] Got: 5
[2] Got: helloworld


In [6]:
from functools import partial

class SequenceNo:
    def __init__(self):
        self.sequence = 0

def handler(result, seq):
    seq.sequence += 1
    print(f'[{seq.sequence}] Got: {result}')

# Create an instance of SequenceNo
seq = SequenceNo()

# Use apply_async with partial to pass extra state
apply_async(add, (2, 3), callback=partial(handler, seq=seq))
apply_async(add, ('hello', 'world'), callback=partial(handler, seq=seq))

[1] Got: 5
[2] Got: helloworld


In [7]:
# Using the same SequenceNo and handler as above
apply_async(add, (2, 3), callback=lambda r: handler(r, seq))
apply_async(add, ('hello', 'world'), callback=lambda r: handler(r, seq))

[3] Got: 5
[4] Got: helloworld
